In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("./dataset_public.csv", index_col=0)

We need to set some constants for the experiment: 
- the search budget $B$ (how many configurations is the optimizer allowed to evaluate), 
- the number of production runs $N$ (how many times will we run the batch workload between subsequent runs of the optimizer), 
- the optimization target (what metric do we want to minimize: `cost` or `runtime`)

In [2]:
B = 33  # search budget
N = 64  # number of production runs
opt_target = 'cost'

When we want to evaluate a cloud configuration for a specific workload, we run `calculate_objective()`, which, for a given dictionary `config`, returns the average value of the optimization target of interest `target`.
- Note that some workloads have been evaluated on some configurations **more than once** - that is why we calculate the average of all found entries.
- Additionally, some workloads with some configurations have **never** been evaluated due to the quotas imposed by some cloud providers - in such cases, the function returns `NaN`.

In [3]:
def calculate_objective(workload, config, target):
    data_w = data[data['workload']==workload]
    matching_entries = data_w.loc[(data_w[list(config)] == pd.Series(config)).all(axis=1)]
    objective_value = matching_entries['target_%s' % (target)].mean()
    return objective_value

After the optimizer has finished the search and has suggested a cloud provider and configuration, we calculate 2 metrics to evaluate the quality of the optimizer's results: regret and savings.

**Regret** expresses the relative percentage difference between the values $f$ of the optimization target for the suggested configuration and for the actually best configuration within the domain:

$Regret = 100 \cdot \frac{f_{found} - f_{best}}{f_{best}}$

**Savings** measure the difference between the total expenses of running the optimizer and of using a random configuration. The expense in this case can be understood as the total monetary cost or runtime, depending on the used optimization target. The savings metric is supposed to evaluate whether the savings achieved by using the optimized configuration makes up for the additional cost of running the optimizer itself. When a **random configuration** is used, the only expense results from $N$ production runs of the workload, each time with the expense $R_{random}$. When we used an **optimized configuration**, there is an additional expense $C_{opt}$ of running the optimizer, which is then followed by $N$ production runs of the workload using the optimized configuration, each time with the expense $R_{opt}$.

$Savings = 100 \cdot \frac{N \cdot R_{random} - (C_{opt} + N \cdot R_{opt})}{N \cdot R_{random}}$

In [4]:
def calculate_results(workload, target, opt_result, opt_expense, n_prod_runs):
    data_w = data[data['workload']==workload]
    true_min = data_w['target_%s' % (target)].min()
    avg_result = data_w['target_%s' % (target)].mean()

    regret = 100 * (opt_result - true_min) / true_min
    savings = 100 * ((n_prod_runs * avg_result) - (opt_expense + n_prod_runs * opt_result)) / (n_prod_runs * avg_result)
    
    return regret, savings

In [5]:
all_regrets = []
all_savings = []
for workload in data['workload'].unique():

    # Perform Random Search
    best_objective = np.inf
    optimization_expense = 0
    evals_made = 0
    np.random.seed(42)
    while evals_made < B:
        # Choose a new point
        new_point = {}
        provider = np.random.choice(data['provider'].unique())
        for feature in data.columns:
            if provider in feature or feature in ['nodes']:
                feature_values = [val for val in data[feature].dropna().unique() if val is not None]
                feature_value = np.random.choice(feature_values)
                new_point[feature] = feature_value

        # Evaluate
        new_objective = calculate_objective(workload=workload,
                                            config=new_point, 
                                            target=opt_target)
        if np.isnan(new_objective):
            continue
        optimization_expense += new_objective
        best_objective = min(best_objective, new_objective)
        evals_made += 1
    
    regret, savings = calculate_results(workload=workload,
                                        target=opt_target,
                                        opt_result=best_objective,
                                        opt_expense=optimization_expense,
                                        n_prod_runs=N)
    all_regrets.append(regret)
    all_savings.append(savings)

print("Average regret:", np.mean(all_regrets))
print("Average savings:", np.mean(all_savings))

Average regret: 25.748349703115505
Average savings: 34.40530063796556
